In [ ]:
from CLEAN.utils import *
ensure_dirs("data/esm_data")
ensure_dirs("data/pretrained")
csv_to_fasta("data/EC2protein_train.csv", "data/EC2protein_train.fasta")

In [ ]:
retrive_esm1b_embedding("EC2protein_train")

### training
might be better to do this outside of the jupyter notebook if it takes a long time

In [ ]:
!./train-supconH.py --training_data EC2protein_train --model_name EC2protein_train_supconH --epoch 1500 --n_pos 9 --n_neg 30 -T 0.1

### inference

In [ ]:
csv_to_fasta("data/test.csv", "data/test.fasta")
retrive_esm1b_embedding("test")

In [ ]:
from CLEAN.infer import infer_pvalue
train_data = "EC2protein_train"
test_data = "test"

infer_pvalue(train_data, test_data, p_value=1e-5, nk_random=20, report_metrics=True, pretrained=True)